## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

## 1. Read in input files
- Read txt fle containing the activity locations 
- Geocode the locations that are contained on the file
- check the CRS (Coordinate Reference System)

In [33]:
# Import modules
import pandas as pd
import geopandas as gpd
import os, sys
from pyproj import CRS

root = r"C:\Users\gcolm\Documents\courses\online\online_courses\others\Geo_python_course\autoGIS_2019\lesson_3\data"
fc = "activity_locations.txt"

data = pd.read_csv(os.path.join(root,fc), sep=';')

data.head()

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
# raise NotImplementedError()

Geocode activity locations

In [23]:
# import module
from geopandas.tools import geocode

# Geocode activty_locations.txt using Nominatim
geo = geocode(data['addr'], provider='nominatim', user_agent='autoGIS_lesson3', timeout=4)

# Print geo
geo.head()


# join data table and geo table, give a name geodata
geodata = None
# Join tables
geodata = geo.join(data)
# Re-order columns
geodata = geodata[['id', 'name', 'address', 'addr', 'geometry']]
# Print geodata
geodata.head()
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
# raise NotImplementedError()

,id,name,address,addr,geometry
0,1000,Helsinki University,"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...","Yliopistonkatu 4, 00100 Helsinki, Finland",POINT (24.94874 60.16972)
1,1001,Ham Helsinki Art Museum,"Helsingin taidemuseo HAM, 8, Eteläinen Rautati...","Eteläinen Rautatiekatu 8, 00100 Helsinki, Finland",POINT (24.93023 60.16978)


In [19]:
# Check coordinate reference system information (re-project/define if needed)
geo.crs
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
# raise NotImplementedError()

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

__Find the closest shopping center for each activity location (closest shopping center from home and closest shopping center from work) and print out the results:__

In [28]:
# Read the shepefile 'shopping_center.shp' 
fc = 'shopping_centers.shp'
sCenters = gpd.read_file(os.path.join(root, fc))

# Print sCenters
sCenters.head()
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
# raise NotImplementedError()

,id,name,address,addr,geometry
0,1000,Itis,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...","Itäkatu 7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,1001,Center,"Helsinki, 1, Kaivokatu, Keskusta, Kluuvi, Etel...","Kaivokatu 1, 00100 Helsinki, Finland",POINT (25496744.578 6673212.851)
2,1002,Iso-omena,"Sports Academy, 11, Piispansilta, Matinkylän k...","Piispansilta 11, 02230 Helsinki, Finland",POINT (25485440.532 6672255.563)
3,1003,Sello,"Mehiläinen Leppävaara, 3, Leppävaarankatu, Sät...","Leppävaarankatu 3, 02600 Helsinki, Finland",POINT (25489707.583 6678342.162)
4,1004,Jumbo,"Robert's Coffee, 3, Valuuttakatu, Vantaanportt...","Valuuttakatu 3, 01510 Helsinki, Finland",POINT (25498056.327 6686627.539)


In [31]:
# Chech sCenters crs and converted to WKT format
sCenters.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - 24.5°E to 25.5°E onshore nominal
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [35]:
# convert to WKT
sCenters.crs = CRS.from_epsg(3879).to_wkt()

In [36]:
# Verify that sCenters crs is now in WKT format
sCenters.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - 24.5°E to 25.5°E onshore nominal
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [37]:
# Check if both datasets have the same crs
geo.crs == sCenters.crs

False

In [38]:
# Change geo crs to be the same as sCenters
geo = geo.to_crs(sCenters.crs)

In [39]:
# Check the crs of address points
print(f'geo CRS infomation:\n{geo.crs}\n')

# Check the crs of population layer
print(f'sCenters CRS infomation:\n{sCenters.crs}')

# Do they match now?
geo.crs == sCenters.crs

geo CRS infomation:
PROJCRS["ETRS89 / GK25FIN",BASEGEOGCRS["ETRS89",DATUM["European Terrestrial Reference System 1989",ELLIPSOID["GRS 1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4258]],CONVERSION["Finland Gauss-Kruger zone 25",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",25,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",1,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",25500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["northing (N)",north,ORDER[1],LENGTHUNIT["metre",1]],AXIS["easting (E)",east,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["Finland - 24.5°E to 25.5°E onshore nominal"],BBOX[59.94,24.5,68.9,2

True

## 2. Create a MutliPoint Object that will contain all of the shopping centers from the shp file

In [26]:
# Import modules
from shapely.geometry import MultiPoint
from shapely.ops import nearest_points

In [54]:
# MultiPoint Object
dest = MultiPoint(list(sCenters['geometry']))

orig1 = geo.at[0, 'geometry']

nearest_geoms = nearest_points(orig1, dest) # Returns a tuple containing the origin point and the nearest destination point
type(nearest_geoms)

for idx, row, in sCenters.iterrows():
    if row['geometry'] == nearest_geoms[1]:
        print(f'Closest shopping center is: {sCenters.at[idx, "name"]}')

Closest shopping center is: Center


## Nearest Point using GeoPandas

To find nearest points from a set of origin points to a set of destination points using GeoDataFrames. 
* datasets to use: `activity_locations.txt` and `shopping_centers.shp` files

SO, for each row of data in the activity_locations -table, we want to figure out the nearest shopping center point and fetch some attributes related to that point. In other words, we want to apply the Shapely `nearest_points`function so that we compare each polygon centroid to all address points, and based on this information access correct attribute information from the address table. 

For doing this, we can create a function that we will apply on the polygon GeoDataFrame:

In [72]:
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""

    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["geometry"].unary_union

    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)

    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]

    nearest_value = nearest_data[value_column].values[0]

    return nearest_value

By default, this function returns the geometry of the nearest point for each row. It is also possible to fetch information from other columns by changing the `value_column` parameter.

In [73]:
geo["nearest_loc"] = geo.apply(get_nearest_values, other_gdf=sCenters, point_column="geometry", axis=1)
geo.head()

,geometry,address,nearest_loc
0,POINT (25497154.366 6672982.716),"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...",POINT (25496744.57755392 6673212.85105319)
1,POINT (25496126.555 6672990.789),"Helsingin taidemuseo HAM, 8, Eteläinen Rautati...",POINT (25496744.57755392 6673212.85105319)


The function creates a MultiPoint object from `other_gdf`  geometry column (in our case, 'sCenters' points) and further passes this MultiPoint object to Shapely's `nearest_points` function. 

In [74]:
geo.apply(get_nearest_values, other_gdf=sCenters, point_column='geometry', value_column="geometry", axis=1)

0    POINT (25496744.57755392 6673212.85105319)
1    POINT (25496744.57755392 6673212.85105319)
dtype: object

- Finally, we can specify that we want the `id` -column for each point, and store the output in a new column `"nearest_loc"`:

In [77]:
geo['nearest_loc'] = geo.apply(get_nearest_values, other_gdf = sCenters, point_column= 'geometry', value_column='name', axis=1)

In [78]:
geo.head()

,geometry,address,nearest_loc
0,POINT (25497154.366 6672982.716),"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...",Center
1,POINT (25496126.555 6672990.789),"Helsingin taidemuseo HAM, 8, Eteläinen Rautati...",Center


Can you think of other application cases for the nearest neighbour analysis?

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()